### Notebooks para analizar las métricas de los forecasts - UNA EJECUCIÓN CADA SEMANA - permite tener mejor idea de las métricas en forecast h=3
Analizar métricas generadas
INPUTS:
- data train - forecast - predicciones fitten en los datos de train (es solo 1 observación)
- data test - predicciones con un horizonte h

In [1]:
import pandas as pd
import numpy as np

from utils.utils import read_processed_data, set_root_path
from mlforecast import MLForecast
import lightgbm as lgb

from mlforecast.lag_transforms import (
    RollingMean,
    SeasonalRollingMean,
)
from sklearn.metrics import mean_absolute_error

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# set root repo
set_root_path()

root path: /Users/joseortega/Documents/GitHub/vn2_challenge


In [3]:
""" 1. read "processed" data """
# leer archivos "processed" generados en step anterior
data, data_state, data_in_stock, data_master, data_submission = (
    read_processed_data(week_index="0", date_index="2024-04-08")
)

data processed readed!


In [ ]:
""" 2. read data forecast OUTPUT BACKTEST """
# obs: los datos de train y test ya tienen el merge con sus datos de true
folder_forecasts = "data/submission/backtest"

data_fcst_real_train_backtest = pd.read_parquet(f"{folder_forecasts}/data_fcst_real_train_backtest.parquet")
data_fcst_real_test_backtest = pd.read_parquet(f"{folder_forecasts}/data_fcst_real_test_backtest.parquet")

In [5]:
data_fcst_real_train_backtest.head(3)

,unique_id,ds,y_true,forecast,forecast_int,week0_update
0,0-126,2021-08-30,5.0,0.950027,1.0,2023-06-05
1,0-126,2021-09-06,0.0,1.683110,2.0,2023-06-05
2,0-126,2021-09-13,0.0,1.757364,2.0,2023-06-05


In [6]:
data_fcst_real_test_backtest.head(3)

,unique_id,ds,y_true,forecast,forecast_int,week0_update
0,0-126,2023-06-12,6.0,4.005469,4.0,2023-06-05
1,0-126,2023-06-19,6.0,3.781803,4.0,2023-06-05
2,0-126,2023-06-26,5.0,3.374413,3.0,2023-06-05


In [7]:
data_fcst_real_train_backtest.shape

(2582223, 6)

In [8]:
data_fcst_real_test_backtest.shape

(75474, 6)

In [10]:
# cantidad de ejecuciones de forecast con semana de inicio distinta, forecast horizonte h=3
data_fcst_real_test_backtest["week0_update"].unique().shape

(42,)

### 5. calcular métricas para cada serie

In [11]:
def calcular_mae_serie(df_metrics, features_columns, column_true, column_pred):
    """
    Calcular métrica MAE a nivel serie
    Interesa saber que series tienen más error, independiente si es un volumen alto o bajo
    (ya que cada producto que quede en stock o faltara stock es un costo asociado)
    """
    df_metrics = df_metrics.copy()

    # agrupar
    df_metrics_output = (
        df_metrics
        .groupby(features_columns)
        .apply(lambda g: pd.Series({
            "mae": mean_absolute_error(g[column_true], g[column_pred]),
            "sum_y_true": g[column_true].sum(),
            "mean_y_true": g[column_true].mean(),
        }))
        .reset_index()
    )

    # ordenar de mayor a menor error (todas las series deben predecirse bien)
    df_metrics_output = df_metrics_output.sort_values("mae", ascending=False)

    return df_metrics_output

In [ ]:
# TRAIN
# el mae más grande es de 5
mae_serie_train = calcular_mae_serie(df_metrics = data_fcst_real_train_backtest, 
                               features_columns = ["unique_id"],
                                column_true = 'y_true',
                                column_pred = 'forecast_int'
                               )
mae_serie_train

/var/folders/fx/0kx46vn52sg544t8x3s4hdl00000gn/T/ipykernel_52405/1876144399.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,unique_id,mae,sum_y_true,mean_y_true
14,19-103,5.148941,38532.0,8.083071
91,60-126,4.855884,119363.0,25.039438
197,61-126,4.331445,96989.0,20.345920
579,64-103,4.274806,56468.0,11.845605
531,63-126,4.025802,51490.0,10.801343
...,...,...,...,...
352,61-43,0.635830,2885.0,0.605202
138,60-270,0.633355,5409.0,1.155029
548,63-213,0.624951,2305.0,0.907123
481,62-268,0.616950,2365.0,0.496119


In [ ]:
# TEST
# SE VE QUE INCREMENTA MUCHO EL MAE. CALCULAR PARA CADA H
mae_serie_test = calcular_mae_serie(df_metrics = data_fcst_real_test_backtest, 
                               features_columns = ["unique_id"],
                                column_true = 'y_true',
                                column_pred = 'forecast_int'
                               )
mae_serie_test

/var/folders/fx/0kx46vn52sg544t8x3s4hdl00000gn/T/ipykernel_52405/1876144399.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,unique_id,mae,sum_y_true,mean_y_true
277,61-23,50.031746,12794.0,101.539683
125,60-23,38.738095,7853.0,62.325397
553,63-23,35.619048,8314.0,65.984127
473,62-23,33.373016,7807.0,61.960317
196,61-124,25.285714,12436.0,98.698413
...,...,...,...,...
442,62-171,0.579365,110.0,0.873016
455,62-20,0.571429,87.0,0.690476
395,61-9,0.563492,94.0,0.746032
482,62-271,0.555556,54.0,0.428571


### 3. metricas test por cada horizonte H=1, H=2, H=3. En todos los horizontes de test hay error

In [20]:
# crear columna horizonte fcst
data_fcst_real_test_backtest["horizonte_fcst"] = (data_fcst_real_test_backtest["ds"] - data_fcst_real_test_backtest["week0_update"]).dt.days / 7

In [26]:
# TEST h=1

# filtrar por h=1
df_aux_filtered = data_fcst_real_test_backtest[data_fcst_real_test_backtest["horizonte_fcst"] == 1]

# calcular mae
mae_serie_test = calcular_mae_serie(df_metrics = df_aux_filtered, 
                               features_columns = ["unique_id"],
                                column_true = 'y_true',
                                column_pred = 'forecast_int'
                               )
mae_serie_test

/var/folders/fx/0kx46vn52sg544t8x3s4hdl00000gn/T/ipykernel_52405/1876144399.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,unique_id,mae,sum_y_true,mean_y_true
277,61-23,52.523810,4219.0,100.452381
125,60-23,35.785714,2590.0,61.666667
553,63-23,35.476190,2790.0,66.428571
473,62-23,29.595238,2594.0,61.761905
196,61-124,24.166667,4172.0,99.333333
...,...,...,...,...
455,62-20,0.571429,29.0,0.690476
482,62-271,0.547619,19.0,0.452381
560,63-278,0.547619,38.0,0.904762
395,61-9,0.523810,32.0,0.761905


In [27]:
# TEST h=2

# filtrar por h=2
df_aux_filtered = data_fcst_real_test_backtest[data_fcst_real_test_backtest["horizonte_fcst"] == 2]

# calcular mae
mae_serie_test = calcular_mae_serie(df_metrics = df_aux_filtered, 
                               features_columns = ["unique_id"],
                                column_true = 'y_true',
                                column_pred = 'forecast_int'
                               )
mae_serie_test

/var/folders/fx/0kx46vn52sg544t8x3s4hdl00000gn/T/ipykernel_52405/1876144399.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,unique_id,mae,sum_y_true,mean_y_true
277,61-23,48.523810,4241.0,100.976190
125,60-23,39.523810,2604.0,62.000000
473,62-23,36.023810,2587.0,61.595238
553,63-23,35.595238,2758.0,65.666667
196,61-124,25.738095,4142.0,98.619048
...,...,...,...,...
376,61-68,0.571429,42.0,1.000000
550,63-218,0.547619,26.0,0.619048
537,63-160,0.547619,22.0,0.523810
442,62-171,0.547619,36.0,0.857143


In [28]:
# TEST h=3

# filtrar por h=3
df_aux_filtered = data_fcst_real_test_backtest[data_fcst_real_test_backtest["horizonte_fcst"] == 3]

# calcular mae
mae_serie_test = calcular_mae_serie(df_metrics = df_aux_filtered, 
                               features_columns = ["unique_id"],
                                column_true = 'y_true',
                                column_pred = 'forecast_int'
                               )
mae_serie_test

/var/folders/fx/0kx46vn52sg544t8x3s4hdl00000gn/T/ipykernel_52405/1876144399.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,unique_id,mae,sum_y_true,mean_y_true
277,61-23,49.047619,4334.0,103.190476
125,60-23,40.904762,2659.0,63.309524
553,63-23,35.785714,2766.0,65.857143
473,62-23,34.500000,2626.0,62.523810
530,63-124,26.452381,3406.0,81.095238
...,...,...,...,...
102,60-169,0.595238,44.0,1.047619
442,62-171,0.571429,37.0,0.880952
482,62-271,0.571429,18.0,0.428571
598,9-49,0.571429,28.0,0.666667


In [30]:
mae_serie_test.head(20)

,unique_id,mae,sum_y_true,mean_y_true
277,61-23,49.047619,4334.0,103.190476
125,60-23,40.904762,2659.0,63.309524
553,63-23,35.785714,2766.0,65.857143
473,62-23,34.500000,2626.0,62.523810
530,63-124,26.452381,3406.0,81.095238
196,61-124,25.952381,4122.0,98.142857
586,64-23,22.928571,1453.0,34.595238
90,60-125,19.547619,3310.0,78.809524
582,64-17,14.857143,2057.0,48.976190
357,61-48,11.976190,1353.0,32.214286


In [32]:
mae_serie_test[mae_serie_test["unique_id"] == "62-191"]

,unique_id,mae,sum_y_true,mean_y_true
450,62-191,0.809524,18.0,0.428571


### 4. Revisar error cuando sub-estima (forecast fue más bajo que el real, por lo que en optimización es un costo de quedarse sin inventario)

In [37]:
# generar columna con forecast - real
data_fcst_real_test_backtest["diff_fcst_real"] = data_fcst_real_test_backtest["forecast_int"] - data_fcst_real_test_backtest["y_true"]
data_fcst_real_test_backtest.head()

,unique_id,ds,y_true,forecast,forecast_int,week0_update,horizonte_fcst,diff_fcst_real
0,0-126,2023-06-12,6.0,4.005469,4.0,2023-06-05,1.0,-2.0
1,0-126,2023-06-19,6.0,3.781803,4.0,2023-06-05,2.0,-2.0
2,0-126,2023-06-26,5.0,3.374413,3.0,2023-06-05,3.0,-2.0
3,0-182,2023-06-12,1.0,1.309988,1.0,2023-06-05,1.0,0.0
4,0-182,2023-06-19,2.0,1.108378,1.0,2023-06-05,2.0,-1.0


In [40]:
# filrar solo cuando se subestima (forecast - real <0)
df_aux_filtered = data_fcst_real_test_backtest[data_fcst_real_test_backtest["diff_fcst_real"] < 0]
df_aux_filtered.head(3)

,unique_id,ds,y_true,forecast,forecast_int,week0_update,horizonte_fcst,diff_fcst_real
0,0-126,2023-06-12,6.0,4.005469,4.0,2023-06-05,1.0,-2.0
1,0-126,2023-06-19,6.0,3.781803,4.0,2023-06-05,2.0,-2.0
2,0-126,2023-06-26,5.0,3.374413,3.0,2023-06-05,3.0,-2.0


In [41]:
# calcular métricas
# calcular mae
mae_serie_test = calcular_mae_serie(df_metrics = df_aux_filtered, 
                               features_columns = ["unique_id"],
                                column_true = 'y_true',
                                column_pred = 'forecast_int'
                               )
mae_serie_test

/var/folders/fx/0kx46vn52sg544t8x3s4hdl00000gn/T/ipykernel_52405/1876144399.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,unique_id,mae,sum_y_true,mean_y_true
277,61-23,55.815789,10195.0,134.144737
125,60-23,48.385965,5728.0,100.491228
553,63-23,45.672414,5619.0,96.879310
473,62-23,38.250000,5497.0,91.616667
196,61-124,32.866667,7549.0,125.816667
...,...,...,...,...
442,62-171,1.000000,45.0,1.956522
482,62-271,1.000000,5.0,1.250000
467,62-217,1.000000,30.0,1.764706
537,63-160,1.000000,9.0,1.500000
